In [1]:
from model import BARTCNN
from MyDataLoader import MyDataset
import pandas as pd
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch.nn as nn
from tqdm import tqdm
import evaluate, torch, time

/home/huynv/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('cuda' if torch.cuda.is_available() else 'cpu')

cuda


In [3]:
df_train = pd.read_csv('/home/huynv/nlp_project/data/vietnews_train.csv').dropna()
df_eval = pd.read_csv('/home/huynv/nlp_project/data/vietnews_test.csv').dropna()

In [4]:
df_train["original"] = df_train["original"].apply(lambda x : x.lower())
df_eval["original"] = df_eval["original"].apply(lambda x : x.lower())
df_train["summary"] = df_train["summary"].apply(lambda x : x.lower())
df_eval["summary"] = df_eval["summary"].apply(lambda x : x.lower())

In [5]:
df_train

,Unnamed: 0,file,original,summary
0,0,vietnews-master/data/train_tokenized\038859.tx...,bộ_trưởng sitharaman phát_biểu tại buổi gặp_gỡ...,bộ_trưởng quốc_phòng ấn_độ bày_tỏ mong_muốn vi...
1,1,vietnews-master/data/train_tokenized\056689.tx...,tổng_thống pháp emmanuel_macron phấn_khích ăn_...,"sau màn ăn_mừng phấn_khích trên khán_đài , emm..."
2,2,vietnews-master/data/train_tokenized\026269.tx...,lực_lượng tuần_duyên nhật_bản đang nỗ_lực tìm_...,một hòn đảo của nhật nằm sát khu_vực tranh_chấ...
3,3,vietnews-master/data/train_tokenized\057161.tx...,"cảnh_sát thành_phố barcelona , tây_ban_nha đã ...",tình_báo mỹ báo với cảnh_sát tây_ban_nha về ng...
4,4,vietnews-master/data/train_tokenized\006464.tx...,"trước_đây , khán_giả thấy rằng nghệ_sĩ xuân_hư...",nghệ_sĩ xuân_hương và mc thanh_bạch đã từng đư...
...,...,...,...,...
105413,105413,vietnews-master/data/train_tokenized\012185.tx...,"dù phải di_chuyển và "" cháy "" ở một lịch_trình...","chiều 16-1 , chương_trình mùa_xuân biển đảo ti..."
105414,105414,vietnews-master/data/train_tokenized\064458.tx...,tại cuộc họp_báo chính_phủ thường_kỳ chiều 4/5...,“ trách_nhiệm cá_nhân hoặc trách_nhiệm đối_với...
105415,105415,vietnews-master/data/train_tokenized\031491.tx...,"ngày hôm_qua 28/2 , tổng_thống francois_hollan...",một tay súng bắn tỉa thuộc lực_lượng cảnh_sát ...
105416,105416,vietnews-master/data/train_tokenized\085659.tx...,do dịp 30-4 và 1-5 rơi vào ngày cuối tuần nên ...,"vietnam_airlines , vietjet , jetstar sẽ tăng t..."


In [6]:
df_eval

,Unnamed: 0,file,original,summary
0,0,vietnews-master/data/test_tokenized\009449.txt...,"đoạn video cho thấy hôm 20/9 , rakeyia_scott đ...",khoảnh_khắc cảnh_sát mỹ bắn một người đàn_ông ...
1,1,vietnews-master/data/test_tokenized\008285.txt...,các binh_sĩ hàn_quốc tham_gia cuộc tập_trận ch...,chi_phí tốn_kém là một trong những lý_do thúc_...
2,2,vietnews-master/data/test_tokenized\002060.txt...,"lên sóng vào năm 2002 , bộ phim giày thuỷ_tinh...",ở tuổi 41 nam chính trong phim “ giày thuỷ_tin...
3,3,vietnews-master/data/test_tokenized\001933.txt...,"kể từ khi công_khai tình_cảm , linh_chi và lâm...","nối lại tình xưa sau thời_gian “ đứt_quãng ” ,..."
4,4,vietnews-master/data/test_tokenized\021360.txt...,"sau khi đẻ rơi trên đường , mẹ con sản_phụ ngu...","đang trên đường đến bệnh_viện bằng xe_máy , ch..."
...,...,...,...,...
22639,22639,vietnews-master/data/test_tokenized\010053.txt...,thủ_tướng nguyễn_xuân_phúc và thủ_tướng nhật_b...,thủ_tướng nhật_bản shinzo_abe hôm_nay cho_biết...
22640,22640,vietnews-master/data/test_tokenized\008629.txt...,thứ_trưởng ngoại_giao syria_faisal al - moqdad...,syria cảnh_báo mọi cuộc tấn_công của mỹ trên l...
22641,22641,vietnews-master/data/test_tokenized\003169.txt...,"nằm trong kế_hoạch di_dời , giải_toả các chung...",ubnd quận 1 cưỡng_chế các hộ dân còn lại tại c...
22642,22642,vietnews-master/data/test_tokenized\004993.txt...,"theo hãng tin reuters , trong các văn_bản vừa ...",bộ ngoại_giao mỹ muốn yêu_cầu tất_cả ứng_viên ...


In [7]:
max_content_length = 1024
max_summary_length = 256
train_dataset = MyDataset(df_train, max_summary_length=max_summary_length, max_content_length=max_content_length)
eval_dataset = MyDataset(df_eval, max_summary_length=max_summary_length, max_content_length=max_content_length)

In [8]:
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=True)

In [9]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [10]:
model = BARTCNN(max_content_length=max_content_length, max_summary_length=max_summary_length).to(device)
# model.apply(initialize_weights)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.NLLLoss()
rouge = evaluate.load('rouge')

In [11]:
n_epochs = 3

In [12]:
def convert_ids_to_sequence(input_ids):
    tokens = model.tokenizer.convert_ids_to_tokens(input_ids)
    line = []
    for token in tokens:
        if token == '<s>':
            line = []
        elif token == '</s>':
            break
        else:
            line.append(str(token).lstrip('▁'))
    return ' '.join(line)

In [13]:
for i in range(n_epochs):
    print(f"Epoch {i + 1}/{n_epochs}")
    total_loss = 0
    count = 0
    t1 = time.time()
    model.train()
    for input_ids, attention_mask, decoder_input_ids, decoder_attention_mask, summary in train_dataloader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        decoder_attention_mask = decoder_attention_mask.to(device)
        decoder_input_ids = decoder_input_ids.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, decoder_input_ids, decoder_attention_mask)

        # remove <pad> in ref
        decoder_input_ids = decoder_input_ids.reshape(-1).to(device)
        indices_not_pad = torch.nonzero(decoder_input_ids != 1).squeeze()
        loss = loss_fn(outputs.reshape(input_ids.shape[0]*max_summary_length, -1)[indices_not_pad], decoder_input_ids[indices_not_pad])

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
        count += 1
        print(f"Batch {count}/{len(train_dataloader)}: Train average loss {total_loss/count}, Time: {time.time() - t1}", end='\r')
    print()

    with torch.no_grad():        
        total_loss = 0
        count = 0
        model.eval()
        eval_ref = []
        eval_predict = []
        t1 = time.time()
        
        for input_ids, attention_mask, decoder_input_ids, decoder_attention_mask, summary in test_dataloader:
            eval_ref.extend(summary)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            decoder_attention_mask = decoder_attention_mask.to(device)
            decoder_input_ids = decoder_input_ids.to(device)

            outputs = model(input_ids, attention_mask, decoder_input_ids, decoder_attention_mask)

            outputs_ids = torch.max(outputs, dim=2).indices.tolist()
            for i in range(len(outputs_ids)):
                eval_predict.append(convert_ids_to_sequence(outputs_ids[i]).lower())

            decoder_input_ids = decoder_input_ids.reshape(-1).to(device)
            indices_not_pad = torch.nonzero(decoder_input_ids != 1).squeeze()
            loss = loss_fn(outputs.reshape(input_ids.shape[0]*max_summary_length, -1)[indices_not_pad], decoder_input_ids[indices_not_pad])
            
            total_loss += loss.item()
            count += 1
            print(f"Batch {count}/{len(test_dataloader)}: Test average loss {total_loss/count}, Time: {time.time() - t1}", end='\r')
    print()
    print(rouge.compute(predictions=eval_predict, references=eval_ref))
    print()

Epoch 1/3


In [ ]:
# Epoch 1/3
# Batch 770/770: Train average loss 5.293656790720952, Time: 1057.91466522216887
# Batch 773/773: Test average loss 6.40927203907541, Time: 979.5249974727631127
# {'rouge1': 0.17748867159262754, 'rouge2': 0.08566437402509883, 'rougeL': 0.15015363559336953, 'rougeLsum': 0.15014160359988066}


In [ ]:
# Epoch 1/3
# Batch 770/770: Train average loss 4.883375053901177, Time: 1074.71507930755629
# Batch 773/773: Test average loss 5.578821538182464, Time: 958.092010259628336
# {'rouge1': 0.05673742266639174, 'rouge2': 0.02629797454664524, 'rougeL': 0.04823451113157266, 'rougeLsum': 0.0482354097017783}

# Epoch 2/3
# Batch 770/770: Train average loss 4.512599832361395, Time: 1111.67359876632768
# Batch 773/773: Test average loss 5.502079916802291, Time: 1028.07295846939094
# {'rouge1': 0.02806078130018447, 'rouge2': 0.013409271469262359, 'rougeL': 0.024761792275180206, 'rougeLsum': 0.02475732321666274}